In [ ]:
import os
from PIL import Image
from glob import glob

# Input paths
image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/train/images"
mask_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/train/masks"

# Output paths
out_image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/images"
out_mask_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/masks"

os.makedirs(out_image_dir, exist_ok=True)
os.makedirs(out_mask_dir, exist_ok=True)

# Supported extensions
exts = ["*.png", "*.jpg", "*.jpeg"]

image_files = []
for ext in exts:
    image_files.extend(glob(os.path.join(image_dir, ext)))

print(f"Found {len(image_files)} images.")

for img_path in image_files:
    fname = os.path.basename(img_path)
    name, ext = os.path.splitext(fname)

    # Load image and mask
    mask_path = os.path.join(mask_dir, fname)
    if not os.path.exists(mask_path):
        print(f"Mask not found for {fname}, skipping.")
        continue

    img = Image.open(img_path)
    mask = Image.open(mask_path)

    # Save original
    img.save(os.path.join(out_image_dir, fname))
    mask.save(os.path.join(out_mask_dir, fname))

    # Horizontal flip
    img_h = img.transpose(Image.FLIP_LEFT_RIGHT)
    mask_h = mask.transpose(Image.FLIP_LEFT_RIGHT)
    img_h.save(os.path.join(out_image_dir, f"{name}_hflip{ext}"))
    mask_h.save(os.path.join(out_mask_dir, f"{name}_hflip{ext}"))

    # Vertical flip
    img_v = img.transpose(Image.FLIP_TOP_BOTTOM)
    mask_v = mask.transpose(Image.FLIP_TOP_BOTTOM)
    img_v.save(os.path.join(out_image_dir, f"{name}_vflip{ext}"))
    mask_v.save(os.path.join(out_mask_dir, f"{name}_vflip{ext}"))

print("Augmentation completed. Files saved in:")
print(out_image_dir)


In [ ]:
#Generate Features for SAM 
import os
import torch
from PIL import Image
from tqdm import tqdm
from transformers import SamModel, SamProcessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/images"  
sam_output_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/sam_train_features"
os.makedirs(sam_output_dir, exist_ok=True)

# Load SAM model for prediction
sam_model = SamModel.from_pretrained("facebook/sam-vit-base").to(device)
sam_processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# === Load image paths ===
image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

# === Inference loop ===
for path in tqdm(image_paths, desc="Extracting SAM masks"):
    img = Image.open(path).convert("RGB")
    base_name = os.path.splitext(os.path.basename(path))[0]

    inputs = sam_processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = sam_model(**inputs)
        masks = outputs.pred_masks  # [B, num_masks, H, W], logits
        iou_scores = outputs.iou_scores  # [B, num_masks]

        # Pick best mask per image 
        best_mask_idx = iou_scores.argmax(dim=1)[0]  # scalar
        best_mask_logits = masks[0, best_mask_idx]    # [H, W], logits

        # Convert logits to soft mask 
        soft_mask = torch.sigmoid(best_mask_logits).cpu()

    # Save soft mask 
    torch.save(soft_mask, os.path.join(sam_output_dir, f"{base_name}.pt"))


In [ ]:
!pip uninstall transformers -y
!pip install transformers==4.40.0

In [2]:
#Generate Features for DINOv2 
import os
import torch
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/images"  
dino_output_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/dino_train_features"
os.makedirs(dino_output_dir, exist_ok=True)

# Load DINOv2 ViT model and processor 
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

# Load image paths
image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

# Inference loop
for path in tqdm(image_paths, desc="Extracting DINOv2 features"):
    img = Image.open(path).convert("RGB")
    base_name = os.path.splitext(os.path.basename(path))[0]

    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state[0]  # [num_tokens, C]

        # Remove CLS token
        spatial_tokens = last_hidden_state[1:]  # [num_patches, C]
        C = spatial_tokens.shape[-1]
        num_patches = spatial_tokens.shape[0]
        H = W = int(num_patches ** 0.5)

        if H * W != num_patches:
            raise ValueError(f"Expected square feature map but got {num_patches} tokens.")

        features = spatial_tokens.reshape(H, W, C).permute(2, 0, 1).contiguous()  # [C, H, W]

    # Save feature map
    torch.save(features.cpu(), os.path.join(dino_output_dir, f"{base_name}.pt"))


/home/deepak1010/anaconda3/envs/sam-vit-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/deepak1010/anaconda3/envs/sam-vit-env/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/deepak1010/anaconda3/envs/sam-vit-env/lib/python3.9/site-packages/torch/cuda/__init__.py:132: UserWarning: 
    Found GPU1 NVIDIA GeForce GT 710 which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))
Ext

In [1]:
!python --version

Python 3.9.22


In [3]:
#Generate Features for SAM for test
import os
import torch
from PIL import Image
from tqdm import tqdm
from transformers import SamModel, SamProcessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_kvasir/images"  # <-- update this
sam_output_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/sam_kvasir_test_features"
os.makedirs(sam_output_dir, exist_ok=True)

# Load SAM model for mask prediction 
sam_model = SamModel.from_pretrained("facebook/sam-vit-base").to(device)
sam_processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# === Load image paths ===
image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

# === Inference loop ===
for path in tqdm(image_paths, desc="Extracting SAM masks"):
    img = Image.open(path).convert("RGB")
    base_name = os.path.splitext(os.path.basename(path))[0]

    inputs = sam_processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = sam_model(**inputs)
        masks = outputs.pred_masks  # [B, num_masks, H, W], logits
        iou_scores = outputs.iou_scores  # [B, num_masks]

        # Pick best mask per image 
        best_mask_idx = iou_scores.argmax(dim=1)[0]  # scalar
        best_mask_logits = masks[0, best_mask_idx]    # [H, W], logits

        # Convert logits to soft mask
        soft_mask = torch.sigmoid(best_mask_logits).cpu()

    # Save soft mask tensor
    torch.save(soft_mask, os.path.join(sam_output_dir, f"{base_name}.pt"))


Extracting SAM masks: 100%|███████████████████████████████| 100/100 [01:21<00:00,  1.23it/s]


In [4]:
#Generate Features for SAM 
import os
import torch
from PIL import Image
from tqdm import tqdm
from transformers import SamModel, SamProcessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_cvc/images"  # <-- update this
sam_output_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/sam_cvc_test_features"
os.makedirs(sam_output_dir, exist_ok=True)

# Load SAM model for mask prediction instead of just embeddings
sam_model = SamModel.from_pretrained("facebook/sam-vit-base").to(device)
sam_processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# === Load image paths ===
image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

# === Inference loop ===
for path in tqdm(image_paths, desc="Extracting SAM masks"):
    img = Image.open(path).convert("RGB")
    base_name = os.path.splitext(os.path.basename(path))[0]

    inputs = sam_processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = sam_model(**inputs)
        masks = outputs.pred_masks  # [B, num_masks, H, W], logits
        iou_scores = outputs.iou_scores  # [B, num_masks]

        # Pick best mask per image (assuming batch size = 1)
        best_mask_idx = iou_scores.argmax(dim=1)[0]  # scalar
        best_mask_logits = masks[0, best_mask_idx]    # [H, W], logits

        # Convert logits to soft mask (sigmoid probabilities)
        soft_mask = torch.sigmoid(best_mask_logits).cpu()

    # Save soft mask tensor
    torch.save(soft_mask, os.path.join(sam_output_dir, f"{base_name}.pt"))


Extracting SAM masks: 100%|█████████████████████████████████| 62/62 [00:49<00:00,  1.24it/s]


In [5]:
#Generate Dinov2 Features for test
import os
import torch
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_kvasir/images"  # Same image dir
dino_output_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/dino_kvasir_test_features"
os.makedirs(dino_output_dir, exist_ok=True)

# Load DINOv2 ViT model and processor
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

# === Load image paths ===
image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

# === Inference loop ===
for path in tqdm(image_paths, desc="Extracting DINOv2 features"):
    img = Image.open(path).convert("RGB")
    base_name = os.path.splitext(os.path.basename(path))[0]

    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state[0]  # [num_tokens, C]
        
        # Remove CLS token
        spatial_tokens = last_hidden_state[1:]  # [num_patches, C]
        C = spatial_tokens.shape[-1]
        num_patches = spatial_tokens.shape[0]
        H = W = int(num_patches ** 0.5)
    
        if H * W != num_patches:
            raise ValueError(f"Expected square feature map but got {num_patches} tokens.")
    
        features = spatial_tokens.reshape(H, W, C).permute(2, 0, 1).contiguous()  # [C, H, W]

    # Save feature map
    torch.save(features.cpu(), os.path.join(dino_output_dir, f"{base_name}.pt"))


Extracting DINOv2 features: 100%|█████████████████████████| 100/100 [00:04<00:00, 23.44it/s]


In [6]:
#Generate Dinov2 Features for test
import os
import torch
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_cvc/images"  # Same image dir
dino_output_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/dino_cvc_test_features"
os.makedirs(dino_output_dir, exist_ok=True)

# Load DINOv2 ViT model and processor
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

# === Load image paths ===
image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

# === Inference loop ===
for path in tqdm(image_paths, desc="Extracting DINOv2 features"):
    img = Image.open(path).convert("RGB")
    base_name = os.path.splitext(os.path.basename(path))[0]

    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state[0]  # [num_tokens, C]
        
        # Remove CLS token
        spatial_tokens = last_hidden_state[1:]  # [num_patches, C]
        C = spatial_tokens.shape[-1]
        num_patches = spatial_tokens.shape[0]
        H = W = int(num_patches ** 0.5)
    
        if H * W != num_patches:
            raise ValueError(f"Expected square feature map but got {num_patches} tokens.")
    
        features = spatial_tokens.reshape(H, W, C).permute(2, 0, 1).contiguous()  # [C, H, W]

    # Save feature map
    torch.save(features.cpu(), os.path.join(dino_output_dir, f"{base_name}.pt"))


Extracting DINOv2 features: 100%|███████████████████████████| 62/62 [00:01<00:00, 32.45it/s]


## In similar manner generate test features for unseen dataset (cvc-300, etis, cvc-colondb) for SAM and DINOv2..............

In [ ]:
#Generate OneFormer Features
import os
import torch
from PIL import Image
from tqdm import tqdm
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_dir = "/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/images"  # update path
oneformer_output_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/oneformer_train_features"
os.makedirs(oneformer_output_dir, exist_ok=True)

# Load OneFormer (COCO semantic segmentation variant)
processor = OneFormerProcessor.from_pretrained("shi-labs/oneformer_coco_swin_large")
model = OneFormerForUniversalSegmentation.from_pretrained("shi-labs/oneformer_coco_swin_large").to(device)

task_type = "semantic"  # can also be "instance" or "panoptic"

image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

for path in tqdm(image_paths, desc="Extracting OneFormer dense features"):
    img = Image.open(path).convert("RGB")
    inputs = processor(images=img, task_inputs=[task_type], return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Dense spatial feature map 
    dense_features = outputs.pixel_decoder_hidden_states[-1]  # shape: [1, C, H, W]
    dense_features = dense_features.squeeze(0).cpu()  # remove batch dim

    base_name = os.path.splitext(os.path.basename(path))[0]
    torch.save(dense_features, os.path.join(oneformer_output_dir, f"{base_name}.pt"))


## Similarly generate all the test features for Oneformer.....................


In [1]:
#Unet++ 3F 
import torch
from torch import nn
import torch.nn.functional as F
import torch.fft

__all__ = ['VGGBlock', 'UNet', 'NestedUNet']


class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out


class NestedUNet(nn.Module):
    def __init__(self, num_classes, input_channels=3, deep_supervision=False,
                 f1_channels=9, f2_channels=768, f3_channels=256):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512]
        self.deep_supervision = deep_supervision

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # --- Standard UNet++ blocks ---
        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2])

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1])

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0])

        # --- Latent space projection layers (L1 / L2) ---
        self.linear1 = nn.Sequential(
            nn.Conv2d(nb_filter[4], 256, 1),
            nn.ReLU()
        )
        self.linear2 = nn.Conv2d(256, 256, 1)

        # --- Fusion layer for external features ---
        in_channels_total = f1_channels + f2_channels + f3_channels
        self.linear = nn.Sequential(
            nn.Conv2d(in_channels_total, 256, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        if self.deep_supervision:
            self.final1 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)

    def forward(self, input, f1, f2, f3):
        # --- Encoder path ---
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))  # bottleneck

        # --- Latent projections ---
        L1 = self.linear1(x4_0)
        L2 = self.linear2(L1)

        # --- Feature alignment & fusion ---
        target_size = f1.shape[2:]
        if f2.shape[2:] != target_size:
            f2 = F.interpolate(f2, size=target_size, mode='bilinear', align_corners=False)
        if f3.shape[2:] != target_size:
            f3 = F.interpolate(f3, size=target_size, mode='bilinear', align_corners=False)

        f = torch.cat([f1, f2, f3], dim=1)
        f = self.linear(f)  # -> (B,256,H,W)

        # --- Frequency filtering ---
        f_freq = torch.fft.fft2(f, norm="ortho")
        f_freq_shifted = torch.fft.fftshift(f_freq)

        B, C, H, W = f_freq_shifted.shape
        low_mask = torch.zeros_like(f_freq_shifted)
        high_mask = torch.ones_like(f_freq_shifted)

        center_h = H // 2
        center_w = W // 2
        radius = min(H, W) // 6

        low_mask[:, :, center_h-radius:center_h+radius, center_w-radius:center_w+radius] = 1
        high_mask = 1 - low_mask

        f_low = torch.real(torch.fft.ifft2(torch.fft.ifftshift(f_freq_shifted * low_mask), norm="ortho"))
        f_high = torch.real(torch.fft.ifft2(torch.fft.ifftshift(f_freq_shifted * high_mask), norm="ortho"))

        if f_low.shape[2:] != L1.shape[2:]:
            f_low = F.interpolate(f_low, size=L1.shape[2:], mode="bilinear", align_corners=False)
        if f_high.shape[2:] != L2.shape[2:]:
            f_high = F.interpolate(f_high, size=L2.shape[2:], mode="bilinear", align_corners=False)

        distillation1 = F.mse_loss(f_low, L1)
        distillation2 = F.mse_loss(f_high, L2)

        # --- Decoder path ---
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4], distillation1, distillation2
        else:
            output = self.final(x0_4)
            return output, distillation1, distillation2


# --- Quick test ---
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = NestedUNet(num_classes=1, input_channels=3).to(device)
    x = torch.randn(1, 3, 352, 352).to(device)
    f1 = torch.randn(1, 9, 32, 32).to(device)       # SAM
    f2 = torch.randn(1, 768, 32, 32).to(device)     # DINOv2
    f3 = torch.randn(1, 256, 32, 32).to(device)     # OneFormer

    out, d1, d2 = model(x, f1, f2, f3)
    print("Output:", out.shape)
    print("Distillation1:", d1.item())
    print("Distillation2:", d2.item())


Output: torch.Size([1, 1, 352, 352])
Distillation1: 0.06858216971158981
Distillation2: 0.06587187945842743


In [2]:
import os
from PIL import Image
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset

class UNetPPWithSAMDINOneFormerDataset(Dataset):
    def __init__(self, image_dir, mask_dir, sam_feature_dir, dino_feature_dir, oneformer_feature_dir, transform=None, feature_size=(64, 64)):
        """
        feature_size: tuple (H, W) → all SAM/DINO/OneFormer features will be resized to this shape
        """
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.sam_feature_dir = sam_feature_dir
        self.dino_feature_dir = dino_feature_dir
        self.oneformer_feature_dir = oneformer_feature_dir
        self.transform = transform
        self.feature_size = feature_size
        self.image_names = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        base_name, _ = os.path.splitext(image_name)

        image_path = os.path.join(self.image_dir, image_name)

        # Find mask file
        mask_path = None
        for ext in [".jpg", ".png"]:
            candidate = os.path.join(self.mask_dir, base_name + ext)
            if os.path.exists(candidate):
                mask_path = candidate
                break
        if mask_path is None:
            raise FileNotFoundError(f"No mask found for {base_name}")

        # Feature paths
        f1_path = os.path.join(self.sam_feature_dir, base_name + ".pt")        # SAM
        f2_path = os.path.join(self.dino_feature_dir, base_name + ".pt")       # DINO
        f3_path = os.path.join(self.oneformer_feature_dir, base_name + ".pt")  # OneFormer

        # Load data
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        f1 = torch.load(f1_path)  # SAM features [N, C, H, W] or [C, H, W]
        f2 = torch.load(f2_path)  # DINO features
        f3 = torch.load(f3_path)  # OneFormer features

        # Ensure 4D shape for interpolation
        if f1.dim() == 3:
            f1 = f1.unsqueeze(0)  # [1, C, H, W]
        if f2.dim() == 3:
            f2 = f2.unsqueeze(0)
        if f3.dim() == 3:
            f3 = f3.unsqueeze(0)

        # Resize features to the same size
        f1 = F.interpolate(f1, size=self.feature_size, mode="bilinear", align_corners=False).squeeze(0)
        f2 = F.interpolate(f2, size=self.feature_size, mode="bilinear", align_corners=False).squeeze(0)
        f3 = F.interpolate(f3, size=self.feature_size, mode="bilinear", align_corners=False).squeeze(0)

        # Apply transforms to image and mask
        if self.transform:
            transformed = self.transform(image=np.array(image), mask=np.array(mask))
            image = transformed['image']
            mask = transformed['mask']

        return {
            "pixel_values": image,
            "ground_truth_mask": mask.float() / 255.0,
            "f1": f1,
            "f2": f2,
            "f3": f3,
            "image_name": image_name
        }


In [4]:
# import albumentations as A
# import os
# from albumentations.pytorch import ToTensorV2
# from torch.utils.data import DataLoader

# # Transform for both train/valid
# transform = A.Compose([
#     A.Resize(352, 352),
#     A.Normalize(),
#     ToTensorV2()
# ])

# train_dataset = UNetWithSAMAndDINODataset(
#     image_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/images",
#     mask_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/masks",
#     sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_train_features",
#     dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_train_features",
#     transform=transform
# )

# test_kvasir_dataset = UNetWithSAMAndDINODataset(
#     image_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_kvasir/images",
#     mask_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_kvasir/masks",
#     sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_kvasir_test_features",
#     dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_kvasir_test_features",
#     transform=transform
# )

# test_cvc_dataset = UNetWithSAMAndDINODataset(
#     image_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_cvc/images",
#     mask_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_cvc/masks",
#     sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_cvc_test_features",
#     dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_cvc_test_features",
#     transform=transform
# )

# test_cvc_300_dataset = UNetWithSAMAndDINODataset(
#     image_dir="/home/deepak1010/Shivanshu Code/CVC-300/images",
#     mask_dir="/home/deepak1010/Shivanshu Code/CVC-300/masks",
#     sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_cvc_300_test_features",
#     dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_cvc_300_test_features",
#     transform=transform
# )

# test_etis_dataset = UNetWithSAMAndDINODataset(
#     image_dir="/home/deepak1010/Shivanshu Code/ETIS/images",
#     mask_dir="/home/deepak1010/Shivanshu Code/ETIS/masks",
#     sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_etis_test_features",
#     dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_etis_test_features",
#     transform=transform
# )

# test_cvc_colondb_dataset = UNetWithSAMAndDINODataset(
#     image_dir="/home/deepak1010/Shivanshu Code/CVC-ColonDB/images",
#     mask_dir="/home/deepak1010/Shivanshu Code/CVC-ColonDB/masks",
#     sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_cvc_colondb_test_features",
#     dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_cvc_colondb_test_features",
#     transform=transform
# )


# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_kvasir_dataloader = DataLoader(test_kvasir_dataset, batch_size=4, shuffle=False)
# test_cvc_dataloader = DataLoader(test_cvc_dataset, batch_size=4, shuffle=False)
# test_cvc_300_dataloader = DataLoader(test_cvc_300_dataset, batch_size=4, shuffle=False)
# test_etis_dataloader = DataLoader(test_etis_dataset, batch_size=4, shuffle=False)
# test_cvc_colondb_dataloader = DataLoader(test_cvc_colondb_dataset, batch_size=4, shuffle=False)


# print(len(train_dataloader))
# print(len(test_kvasir_dataloader))
# print(len(test_cvc_dataloader))
# print(len(test_cvc_300_dataloader))
# print(len(test_etis_dataloader))
# print(len(test_cvc_colondb_dataloader))

1450
25
16
15
49
95


/home/deepak1010/anaconda3/envs/sam-vit-env/lib/python3.9/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
import albumentations as A
import os
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# Transform for both train/valid
transform = A.Compose([
    A.Resize(352, 352),
    A.Normalize(),
    ToTensorV2()
])

train_dataset = UNetPPWithSAMDINOneFormerDataset(
    image_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/images",
    mask_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc_augmented/train/masks",
    sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_train_features",
    dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_train_features",
    oneformer_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/oneformer_train_features",
    transform=transform
)

test_kvasir_dataset = UNetPPWithSAMDINOneFormerDataset(
    image_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_kvasir/images",
    mask_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_kvasir/masks",
    sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_kvasir_test_features",
    dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_kvasir_test_features",
    oneformer_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/oneformer_kvasir_test_features",
    transform=transform
)

test_cvc_dataset = UNetPPWithSAMDINOneFormerDataset(
    image_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_cvc/images",
    mask_dir="/home/deepak1010/Shivanshu Code/mixed_ds_kvasir_cvc/test_cvc/masks",
    sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_cvc_test_features",
    dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_cvc_test_features",
    oneformer_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/oneformer_cvc_test_features",
    transform=transform
)

test_cvc_300_dataset = UNetPPWithSAMDINOneFormerDataset(
    image_dir="/home/deepak1010/Shivanshu Code/CVC-300/images",
    mask_dir="/home/deepak1010/Shivanshu Code/CVC-300/masks",
    sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_cvc_300_test_features",
    dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_cvc_300_test_features",
    oneformer_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/oneformer_cvc_300_test_features",
    transform=transform
)

test_etis_dataset = UNetPPWithSAMDINOneFormerDataset(
    image_dir="/home/deepak1010/Shivanshu Code/ETIS/images",
    mask_dir="/home/deepak1010/Shivanshu Code/ETIS/masks",
    sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_etis_test_features",
    dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_etis_test_features",
    oneformer_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/oneformer_etis_test_features",
    transform=transform
)

test_cvc_colondb_dataset = UNetPPWithSAMDINOneFormerDataset(
    image_dir="/home/deepak1010/Shivanshu Code/CVC-ColonDB/images",
    mask_dir="/home/deepak1010/Shivanshu Code/CVC-ColonDB/masks",
    sam_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/sam_cvc_colondb_test_features",
    dino_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/dino_cvc_colondb_test_features",
    oneformer_feature_dir="/home/deepak1010/Shivanshu Code/features_sam_clip/oneformer_cvc_colondb_test_features",
    transform=transform
)

# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)
test_kvasir_dataloader = DataLoader(test_kvasir_dataset, batch_size=6, shuffle=False)
test_cvc_dataloader = DataLoader(test_cvc_dataset, batch_size=6, shuffle=False)
test_cvc_300_dataloader = DataLoader(test_cvc_300_dataset, batch_size=6, shuffle=False)
test_etis_dataloader = DataLoader(test_etis_dataset, batch_size=6, shuffle=False)
test_cvc_colondb_dataloader = DataLoader(test_cvc_colondb_dataset, batch_size=6, shuffle=False)

# Quick check
print(len(train_dataloader))
print(len(test_kvasir_dataloader))
print(len(test_cvc_dataloader))
print(len(test_cvc_300_dataloader))
print(len(test_etis_dataloader))
print(len(test_cvc_colondb_dataloader))


967
17
11
10
33
64


In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
from statistics import mean
from PIL import Image

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)
        preds = preds.clamp(min=1e-7, max=1 - 1e-7)
        preds = preds.contiguous()
        targets = targets.contiguous()

        intersection = (preds * targets).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets.sum(dim=(2, 3))

        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

# === Device, model, optimizer ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NestedUNet(num_classes=1, input_channels=3).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
seg_loss = DiceLoss()
criteria = nn.BCEWithLogitsLoss()

# === Training settings ===
num_epochs = 75
best_train_loss = float("inf")
patience = 10
early_stop_counter = 0
min_delta = 1e-4

for epoch in range(num_epochs):
    model.train()
    epoch_losses, seg_losses, distil_losses = [], [], []

    if epoch == 51:
        print("Freezing encoder layers (conv0_0 to conv4_0)...")
        for name, param in model.named_parameters():
            if any(enc in name for enc in ['conv0_0', 'conv1_0', 'conv2_0', 'conv3_0', 'conv4_0']):
                param.requires_grad = False
                print(f"Frozen: {name}")

    tq = tqdm(train_dataloader, desc=f"[Train] Epoch {epoch}")
    for batch in tq:
        x = batch["pixel_values"].to(device)                # Input image
        mask = batch["ground_truth_mask"].unsqueeze(1).to(device)

        # SAM features
        f1 = batch["f1"].to(device)                         # Shape: [B, N, C, H, W]
        B, N, C, H, W = f1.shape
        f1 = f1.view(B, N * C, H, W)                         # → [B, 512, H, W] if N=1, C=512

        # DINOv2 features
        f2 = batch["f2"].to(device)                         # Shape: [B, 512, H, W] 
        #Oneformer Features
        f3 = batch["f3"].to(device)
        # print(f1.shape)
        # print(f2.shape)
        # print(f3.shape)
        logits, dist1, dist2 = model(x, f1, f2, f3)

        # Resize mask to match logits
        mask_resized = nn.functional.interpolate(mask, size=logits.shape[2:], mode="bilinear", align_corners=False)

        segmentation_loss = seg_loss(logits, mask_resized) + criteria(logits, mask_resized)

        if epoch < 25:
            loss = segmentation_loss
        else:
            loss = 0.6 * segmentation_loss + 0.1 * dist1 + 0.1 * dist2

        if not torch.isnan(loss):
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
            seg_losses.append(segmentation_loss.item())
            distil_losses.append((dist1.item() + dist2.item()) / 2)

        tq.set_postfix({
            "Loss": loss.item(),
            "Seg": segmentation_loss.item(),
            "Dist": (dist1.item() + dist2.item()) / 2
        })

    avg_train_loss = mean(epoch_losses)
    print(f"[Train] Epoch {epoch:03}: Loss = {avg_train_loss:.4f}, Seg = {mean(seg_losses):.4f}, Dist = {mean(distil_losses):.4f}")

    # === Early Stopping Check ===
    if best_train_loss - avg_train_loss > min_delta:
        best_train_loss = avg_train_loss
        early_stop_counter = 0
        save_path = "/home/deepak1010/Shivanshu Code/features_sam_clip/unetplusplus_sam_dino_oneformer.pth"
        torch.save(model.state_dict(), save_path)
        print(f"✅ Saved new best model at epoch {epoch} with train_loss = {best_train_loss:.4f}")
    else:
        early_stop_counter += 1
        print(f"Early stopping counter: {early_stop_counter}/{patience}")
        if early_stop_counter >= patience:
            print(" Early stopping triggered. Stopping training.")
            break


[Train] Epoch 0: 100%|█| 967/967 [24:56<00:00,  1.55s/it, Loss=0.834, Seg=0.834, Dist=0.181]


[Train] Epoch 000: Loss = 1.0143, Seg = 1.0143, Dist = 0.1807
✅ Saved new best model at epoch 0 with train_loss = 1.0143


[Train] Epoch 1: 100%|█| 967/967 [22:10<00:00,  1.38s/it, Loss=0.652, Seg=0.652, Dist=0.179]


[Train] Epoch 001: Loss = 0.6308, Seg = 0.6308, Dist = 0.1778
✅ Saved new best model at epoch 1 with train_loss = 0.6308


[Train] Epoch 2: 100%|████| 967/967 [22:00<00:00,  1.37s/it, Loss=0.28, Seg=0.28, Dist=0.18]


...

[Train] Epoch 72: 100%|█| 967/967 [17:00<00:00,  1.06s/it, Loss=0.0137, Seg=0.0229, Dist=7.6


[Train] Epoch 072: Loss = 0.0149, Seg = 0.0249, Dist = 0.0000
Early stopping counter: 1/10


[Train] Epoch 73: 100%|█| 967/967 [17:01<00:00,  1.06s/it, Loss=0.0127, Seg=0.0211, Dist=4.3


[Train] Epoch 073: Loss = 0.0142, Seg = 0.0237, Dist = 0.0000
✅ Saved new best model at epoch 73 with train_loss = 0.0142


[Train] Epoch 74:   7%| | 70/967 [01:13<16:09,  1.08s/it, Loss=0.0144, Seg=0.0239, Dist=5.12

In [4]:
# === Testing for Kvasir with SAM + DINO + OneFormer Features ===
import torch
import torch.nn as nn
from tqdm import tqdm
import os
from statistics import mean
from torchvision.utils import save_image

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)
        preds = preds.clamp(min=1e-7, max=1 - 1e-7)
        preds = preds.contiguous()
        targets = targets.contiguous()

        intersection = (preds * targets).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets.sum(dim=(2, 3))

        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

def compute_metrics(preds, targets, smooth=1e-6):
    preds = preds.view(-1)
    targets = targets.view(-1)

    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()

    dice = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    iou = (tp + smooth) / (tp + fp + fn + smooth)
    precision = (tp + smooth) / (tp + fp + smooth)
    recall = (tp + smooth) / (tp + fn + smooth)

    return dice, iou, precision, recall

# ---- DEVICE ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- LOAD MODEL ----
model = NestedUNet(num_classes=1, input_channels=3).to(device)
model.load_state_dict(torch.load(
    "/home/deepak1010/Shivanshu Code/features_sam_clip/unetplusplus_sam_dino_oneformer.pth",
    map_location=device
))
model.eval()

# ---- LOSSES ----
seg_loss = DiceLoss()
criteria = nn.BCEWithLogitsLoss()

# ---- TEST LOOP ----
test_losses, test_seg_losses, test_distil_losses = [], [], []
all_dice, all_iou, all_precision, all_recall = [], [], [], []

save_pred_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_kvasir_unetplusplus"
os.makedirs(save_pred_dir, exist_ok=True)

with torch.no_grad():
    tq = tqdm(test_kvasir_dataloader, desc="[Test]")
    for i, batch in enumerate(tq):
        # Input image & GT
        x = batch["pixel_values"].to(device)
        mask = batch["ground_truth_mask"].unsqueeze(1).to(device)

        # SAM features
        f1 = batch["f1"].to(device)                         # [B, N, C, H, W]
        B, N, C, H, W = f1.shape
        f1 = f1.view(B, N * C, H, W)                        # [B, N*C, H, W]

        # DINOv2 features
        f2 = batch["f2"].to(device)                         # [B, 512, H, W]

        # OneFormer features
        f3 = batch["f3"].to(device)                         # shape depends on extractor

        # Forward pass
        logits, dist1, dist2 = model(x, f1, f2, f3)

        # Resize mask
        mask_resized = nn.functional.interpolate(mask, size=logits.shape[2:], mode="bilinear", align_corners=False)

        # Loss calculation (same as training)
        seg_loss_value = seg_loss(logits, mask_resized) + criteria(logits, mask_resized)
        loss = 0.6 * seg_loss_value + 0.1 * dist1 + 0.1 * dist2

        test_losses.append(loss.item())
        test_seg_losses.append(seg_loss_value.item())
        test_distil_losses.append((dist1.item() + dist2.item()) / 2)

        # Predictions -> binary
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()

        # Metrics
        dice, iou, precision, recall = compute_metrics(preds.cpu(), mask_resized.cpu())
        all_dice.append(dice)
        all_iou.append(iou)
        all_precision.append(precision)
        all_recall.append(recall)

        # Save first 20 predictions
        if i < 20:
            save_image(preds, os.path.join(save_pred_dir, f"pred_{i}.png"))
            save_image(mask_resized.float(), os.path.join(save_pred_dir, f"gt_{i}.png"))

# ---- FINAL RESULTS ----
print(f"[Test Results] Loss = {mean(test_losses):.4f}, Seg = {mean(test_seg_losses):.4f}, Dist = {mean(test_distil_losses):.4f}")
print(f"[Metrics] mDice = {mean(all_dice):.4f}, mIoU = {mean(all_iou):.4f}, Precision = {mean(all_precision):.4f}, Recall = {mean(all_recall):.4f}")
print(f"✅ Predictions saved in {save_pred_dir}")


[Test]: 100%|███████████████████████████████████████████████| 17/17 [00:20<00:00,  1.22s/it]

[Test Results] Loss = 0.3000, Seg = 0.5000, Dist = 0.0000
[Metrics] mDice = 0.8586, mIoU = 0.7584, Precision = 0.8831, Recall = 0.8467
✅ Predictions saved in /home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_kvasir_unetplusplus


In [5]:
# === Testing for Kvasir with SAM + DINO + OneFormer Features ===
import torch
import torch.nn as nn
from tqdm import tqdm
import os
from statistics import mean
from torchvision.utils import save_image

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)
        preds = preds.clamp(min=1e-7, max=1 - 1e-7)
        preds = preds.contiguous()
        targets = targets.contiguous()

        intersection = (preds * targets).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets.sum(dim=(2, 3))

        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

def compute_metrics(preds, targets, smooth=1e-6):
    preds = preds.view(-1)
    targets = targets.view(-1)

    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()

    dice = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    iou = (tp + smooth) / (tp + fp + fn + smooth)
    precision = (tp + smooth) / (tp + fp + smooth)
    recall = (tp + smooth) / (tp + fn + smooth)

    return dice, iou, precision, recall

# ---- DEVICE ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- LOAD MODEL ----
model = NestedUNet(num_classes=1, input_channels=3).to(device)
model.load_state_dict(torch.load(
    "/home/deepak1010/Shivanshu Code/features_sam_clip/unetplusplus_sam_dino_oneformer.pth",
    map_location=device
))
model.eval()

# ---- LOSSES ----
seg_loss = DiceLoss()
criteria = nn.BCEWithLogitsLoss()

# ---- TEST LOOP ----
test_losses, test_seg_losses, test_distil_losses = [], [], []
all_dice, all_iou, all_precision, all_recall = [], [], [], []

save_pred_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_cvc_unetplusplus"
os.makedirs(save_pred_dir, exist_ok=True)

with torch.no_grad():
    tq = tqdm(test_cvc_dataloader, desc="[Test]")
    for i, batch in enumerate(tq):
        # Input image & GT
        x = batch["pixel_values"].to(device)
        mask = batch["ground_truth_mask"].unsqueeze(1).to(device)

        # SAM features
        f1 = batch["f1"].to(device)                         # [B, N, C, H, W]
        B, N, C, H, W = f1.shape
        f1 = f1.view(B, N * C, H, W)                        # [B, N*C, H, W]

        # DINOv2 features
        f2 = batch["f2"].to(device)                         # [B, 512, H, W]

        # OneFormer features
        f3 = batch["f3"].to(device)                         # shape depends on extractor

        # Forward pass
        logits, dist1, dist2 = model(x, f1, f2, f3)

        # Resize mask
        mask_resized = nn.functional.interpolate(mask, size=logits.shape[2:], mode="bilinear", align_corners=False)

        # Loss calculation (same as training)
        seg_loss_value = seg_loss(logits, mask_resized) + criteria(logits, mask_resized)
        loss = 0.6 * seg_loss_value + 0.1 * dist1 + 0.1 * dist2

        test_losses.append(loss.item())
        test_seg_losses.append(seg_loss_value.item())
        test_distil_losses.append((dist1.item() + dist2.item()) / 2)

        # Predictions -> binary
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()

        # Metrics
        dice, iou, precision, recall = compute_metrics(preds.cpu(), mask_resized.cpu())
        all_dice.append(dice)
        all_iou.append(iou)
        all_precision.append(precision)
        all_recall.append(recall)

        # Save first 20 predictions
        if i < 20:
            save_image(preds, os.path.join(save_pred_dir, f"pred_{i}.png"))
            save_image(mask_resized.float(), os.path.join(save_pred_dir, f"gt_{i}.png"))

# ---- FINAL RESULTS ----
print(f"[Test Results] Loss = {mean(test_losses):.4f}, Seg = {mean(test_seg_losses):.4f}, Dist = {mean(test_distil_losses):.4f}")
print(f"[Metrics] mDice = {mean(all_dice):.4f}, mIoU = {mean(all_iou):.4f}, Precision = {mean(all_precision):.4f}, Recall = {mean(all_recall):.4f}")
print(f"✅ Predictions saved in {save_pred_dir}")


[Test]: 100%|███████████████████████████████████████████████| 11/11 [00:12<00:00,  1.09s/it]

[Test Results] Loss = 0.0684, Seg = 0.1139, Dist = 0.0000
[Metrics] mDice = 0.9471, mIoU = 0.9000, Precision = 0.9563, Recall = 0.9387
✅ Predictions saved in /home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_cvc_unetplusplus


In [6]:
# === Testing for Kvasir with SAM + DINO + OneFormer Features ===
import torch
import torch.nn as nn
from tqdm import tqdm
import os
from statistics import mean
from torchvision.utils import save_image

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)
        preds = preds.clamp(min=1e-7, max=1 - 1e-7)
        preds = preds.contiguous()
        targets = targets.contiguous()

        intersection = (preds * targets).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets.sum(dim=(2, 3))

        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

def compute_metrics(preds, targets, smooth=1e-6):
    preds = preds.view(-1)
    targets = targets.view(-1)

    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()

    dice = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    iou = (tp + smooth) / (tp + fp + fn + smooth)
    precision = (tp + smooth) / (tp + fp + smooth)
    recall = (tp + smooth) / (tp + fn + smooth)

    return dice, iou, precision, recall

# ---- DEVICE ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- LOAD MODEL ----
model = NestedUNet(num_classes=1, input_channels=3).to(device)
model.load_state_dict(torch.load(
    "/home/deepak1010/Shivanshu Code/features_sam_clip/unetplusplus_sam_dino_oneformer.pth",
    map_location=device
))
model.eval()

# ---- LOSSES ----
seg_loss = DiceLoss()
criteria = nn.BCEWithLogitsLoss()

# ---- TEST LOOP ----
test_losses, test_seg_losses, test_distil_losses = [], [], []
all_dice, all_iou, all_precision, all_recall = [], [], [], []

save_pred_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_cvc_300_unetplusplus"
os.makedirs(save_pred_dir, exist_ok=True)

with torch.no_grad():
    tq = tqdm(test_cvc_300_dataloader, desc="[Test]")
    for i, batch in enumerate(tq):
        # Input image & GT
        x = batch["pixel_values"].to(device)
        mask = batch["ground_truth_mask"].unsqueeze(1).to(device)

        # SAM features
        f1 = batch["f1"].to(device)                         # [B, N, C, H, W]
        B, N, C, H, W = f1.shape
        f1 = f1.view(B, N * C, H, W)                        # [B, N*C, H, W]

        # DINOv2 features
        f2 = batch["f2"].to(device)                         # [B, 512, H, W]

        # OneFormer features
        f3 = batch["f3"].to(device)                         # shape depends on extractor

        # Forward pass
        logits, dist1, dist2 = model(x, f1, f2, f3)

        # Resize mask
        mask_resized = nn.functional.interpolate(mask, size=logits.shape[2:], mode="bilinear", align_corners=False)

        # Loss calculation (same as training)
        seg_loss_value = seg_loss(logits, mask_resized) + criteria(logits, mask_resized)
        loss = 0.6 * seg_loss_value + 0.1 * dist1 + 0.1 * dist2

        test_losses.append(loss.item())
        test_seg_losses.append(seg_loss_value.item())
        test_distil_losses.append((dist1.item() + dist2.item()) / 2)

        # Predictions -> binary
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()

        # Metrics
        dice, iou, precision, recall = compute_metrics(preds.cpu(), mask_resized.cpu())
        all_dice.append(dice)
        all_iou.append(iou)
        all_precision.append(precision)
        all_recall.append(recall)

        # Save first 20 predictions
        if i < 20:
            save_image(preds, os.path.join(save_pred_dir, f"pred_{i}.png"))
            save_image(mask_resized.float(), os.path.join(save_pred_dir, f"gt_{i}.png"))

# ---- FINAL RESULTS ----
print(f"[Test Results] Loss = {mean(test_losses):.4f}, Seg = {mean(test_seg_losses):.4f}, Dist = {mean(test_distil_losses):.4f}")
print(f"[Metrics] mDice = {mean(all_dice):.4f}, mIoU = {mean(all_iou):.4f}, Precision = {mean(all_precision):.4f}, Recall = {mean(all_recall):.4f}")
print(f"✅ Predictions saved in {save_pred_dir}")


[Test]: 100%|███████████████████████████████████████████████| 10/10 [00:10<00:00,  1.09s/it]

[Test Results] Loss = 0.1512, Seg = 0.2520, Dist = 0.0000
[Metrics] mDice = 0.8416, mIoU = 0.7447, Precision = 0.8565, Recall = 0.8399
✅ Predictions saved in /home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_cvc_300_unetplusplus


In [7]:
# === Testing for Kvasir with SAM + DINO + OneFormer Features ===
import torch
import torch.nn as nn
from tqdm import tqdm
import os
from statistics import mean
from torchvision.utils import save_image

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)
        preds = preds.clamp(min=1e-7, max=1 - 1e-7)
        preds = preds.contiguous()
        targets = targets.contiguous()

        intersection = (preds * targets).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets.sum(dim=(2, 3))

        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

def compute_metrics(preds, targets, smooth=1e-6):
    preds = preds.view(-1)
    targets = targets.view(-1)

    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()

    dice = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    iou = (tp + smooth) / (tp + fp + fn + smooth)
    precision = (tp + smooth) / (tp + fp + smooth)
    recall = (tp + smooth) / (tp + fn + smooth)

    return dice, iou, precision, recall

# ---- DEVICE ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- LOAD MODEL ----
model = NestedUNet(num_classes=1, input_channels=3).to(device)
model.load_state_dict(torch.load(
    "/home/deepak1010/Shivanshu Code/features_sam_clip/unetplusplus_sam_dino_oneformer.pth",
    map_location=device
))
model.eval()

# ---- LOSSES ----
seg_loss = DiceLoss()
criteria = nn.BCEWithLogitsLoss()

# ---- TEST LOOP ----
test_losses, test_seg_losses, test_distil_losses = [], [], []
all_dice, all_iou, all_precision, all_recall = [], [], [], []

save_pred_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_cvc_colondb_unetplusplus"
os.makedirs(save_pred_dir, exist_ok=True)

with torch.no_grad():
    tq = tqdm(test_cvc_colondb_dataloader, desc="[Test]")
    for i, batch in enumerate(tq):
        # Input image & GT
        x = batch["pixel_values"].to(device)
        mask = batch["ground_truth_mask"].unsqueeze(1).to(device)

        # SAM features
        f1 = batch["f1"].to(device)                         # [B, N, C, H, W]
        B, N, C, H, W = f1.shape
        f1 = f1.view(B, N * C, H, W)                        # [B, N*C, H, W]

        # DINOv2 features
        f2 = batch["f2"].to(device)                         # [B, 512, H, W]

        # OneFormer features
        f3 = batch["f3"].to(device)                         # shape depends on extractor

        # Forward pass
        logits, dist1, dist2 = model(x, f1, f2, f3)

        # Resize mask
        mask_resized = nn.functional.interpolate(mask, size=logits.shape[2:], mode="bilinear", align_corners=False)

        # Loss calculation (same as training)
        seg_loss_value = seg_loss(logits, mask_resized) + criteria(logits, mask_resized)
        loss = 0.6 * seg_loss_value + 0.1 * dist1 + 0.1 * dist2

        test_losses.append(loss.item())
        test_seg_losses.append(seg_loss_value.item())
        test_distil_losses.append((dist1.item() + dist2.item()) / 2)

        # Predictions -> binary
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()

        # Metrics
        dice, iou, precision, recall = compute_metrics(preds.cpu(), mask_resized.cpu())
        all_dice.append(dice)
        all_iou.append(iou)
        all_precision.append(precision)
        all_recall.append(recall)

        # Save first 20 predictions
        if i < 20:
            save_image(preds, os.path.join(save_pred_dir, f"pred_{i}.png"))
            save_image(mask_resized.float(), os.path.join(save_pred_dir, f"gt_{i}.png"))

# ---- FINAL RESULTS ----
print(f"[Test Results] Loss = {mean(test_losses):.4f}, Seg = {mean(test_seg_losses):.4f}, Dist = {mean(test_distil_losses):.4f}")
print(f"[Metrics] mDice = {mean(all_dice):.4f}, mIoU = {mean(all_iou):.4f}, Precision = {mean(all_precision):.4f}, Recall = {mean(all_recall):.4f}")
print(f"✅ Predictions saved in {save_pred_dir}")


[Test]: 100%|███████████████████████████████████████████████| 64/64 [01:12<00:00,  1.14s/it]

[Test Results] Loss = 0.5373, Seg = 0.8955, Dist = 0.0000
[Metrics] mDice = 0.7115, mIoU = 0.5987, Precision = 0.8333, Recall = 0.6787
✅ Predictions saved in /home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_cvc_colondb_unetplusplus


In [8]:
# === Testing for Kvasir with SAM + DINO + OneFormer Features ===
import torch
import torch.nn as nn
from tqdm import tqdm
import os
from statistics import mean
from torchvision.utils import save_image

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = torch.sigmoid(preds)
        preds = preds.clamp(min=1e-7, max=1 - 1e-7)
        preds = preds.contiguous()
        targets = targets.contiguous()

        intersection = (preds * targets).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets.sum(dim=(2, 3))

        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

def compute_metrics(preds, targets, smooth=1e-6):
    preds = preds.view(-1)
    targets = targets.view(-1)

    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()

    dice = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    iou = (tp + smooth) / (tp + fp + fn + smooth)
    precision = (tp + smooth) / (tp + fp + smooth)
    recall = (tp + smooth) / (tp + fn + smooth)

    return dice, iou, precision, recall

# ---- DEVICE ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- LOAD MODEL ----
model = NestedUNet(num_classes=1, input_channels=3).to(device)
model.load_state_dict(torch.load(
    "/home/deepak1010/Shivanshu Code/features_sam_clip/unetplusplus_sam_dino_oneformer.pth",
    map_location=device
))
model.eval()

# ---- LOSSES ----
seg_loss = DiceLoss()
criteria = nn.BCEWithLogitsLoss()

# ---- TEST LOOP ----
test_losses, test_seg_losses, test_distil_losses = [], [], []
all_dice, all_iou, all_precision, all_recall = [], [], [], []

save_pred_dir = "/home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_etis_unetplusplus"
os.makedirs(save_pred_dir, exist_ok=True)

with torch.no_grad():
    tq = tqdm(test_etis_dataloader, desc="[Test]")
    for i, batch in enumerate(tq):
        # Input image & GT
        x = batch["pixel_values"].to(device)
        mask = batch["ground_truth_mask"].unsqueeze(1).to(device)

        # SAM features
        f1 = batch["f1"].to(device)                         # [B, N, C, H, W]
        B, N, C, H, W = f1.shape
        f1 = f1.view(B, N * C, H, W)                        # [B, N*C, H, W]

        # DINOv2 features
        f2 = batch["f2"].to(device)                         # [B, 512, H, W]

        # OneFormer features
        f3 = batch["f3"].to(device)                         # shape depends on extractor

        # Forward pass
        logits, dist1, dist2 = model(x, f1, f2, f3)

        # Resize mask
        mask_resized = nn.functional.interpolate(mask, size=logits.shape[2:], mode="bilinear", align_corners=False)

        # Loss calculation (same as training)
        seg_loss_value = seg_loss(logits, mask_resized) + criteria(logits, mask_resized)
        loss = 0.6 * seg_loss_value + 0.1 * dist1 + 0.1 * dist2

        test_losses.append(loss.item())
        test_seg_losses.append(seg_loss_value.item())
        test_distil_losses.append((dist1.item() + dist2.item()) / 2)

        # Predictions -> binary
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()

        # Metrics
        dice, iou, precision, recall = compute_metrics(preds.cpu(), mask_resized.cpu())
        all_dice.append(dice)
        all_iou.append(iou)
        all_precision.append(precision)
        all_recall.append(recall)

        # Save first 20 predictions
        if i < 20:
            save_image(preds, os.path.join(save_pred_dir, f"pred_{i}.png"))
            save_image(mask_resized.float(), os.path.join(save_pred_dir, f"gt_{i}.png"))

# ---- FINAL RESULTS ----
print(f"[Test Results] Loss = {mean(test_losses):.4f}, Seg = {mean(test_seg_losses):.4f}, Dist = {mean(test_distil_losses):.4f}")
print(f"[Metrics] mDice = {mean(all_dice):.4f}, mIoU = {mean(all_iou):.4f}, Precision = {mean(all_precision):.4f}, Recall = {mean(all_recall):.4f}")
print(f"✅ Predictions saved in {save_pred_dir}")


[Test]: 100%|███████████████████████████████████████████████| 33/33 [00:52<00:00,  1.60s/it]

[Test Results] Loss = 0.4775, Seg = 0.7959, Dist = 0.0000
[Metrics] mDice = 0.5188, mIoU = 0.3992, Precision = 0.6649, Recall = 0.5040
✅ Predictions saved in /home/deepak1010/Shivanshu Code/features_sam_clip/test_predictions_etis_unetplusplus
